In [1]:
import pandas as pd 

In [2]:
!pip install tensorflow

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
# Remove space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1            CONFIRMED
2       False_Positive
3       False_Positive
4            CONFIRMED
5            CONFIRMED
6            CONFIRMED
7            CONFIRMED
8       False_Positive
9            CONFIRMED
10           CONFIRMED
11           CONFIRMED
12           CONFIRMED
13           CONFIRMED
14      False_Positive
15      False_Positive
16      False_Positive
17      False_Positive
18           CONFIRMED
19           CONFIRMED
20      False_Positive
21           CONFIRMED
22           CONFIRMED
23           CONFIRMED
24      False_Positive
25           CONFIRMED
26           CONFIRMED
27           CONFIRMED
28      False_Positive
29      False_Positive
             ...      
9532    False_Positive
9533         CANDIDATE
9535    False_Positive
9536    False_Positive
9537    False_Positive
9538    False_Positive
9539         CANDIDATE
9540         CONFIRMED
9541    False_Positive
9542         CANDIDATE
9543    False_Positive
9544         CANDIDATE
9545    Fal

# Create a Train Test Split
Use `koi_disposition` for the y values

In [5]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]

X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3206,0,1,1,0,31.805143,0.000042,-0.000042,186.38905,0.00105,-0.00105,...,-161.0,4.545,0.044,-0.176,0.863,0.217,-0.072,298.04453,40.086361,14.517
3954,0,0,0,0,24.560711,0.000375,-0.000375,146.79380,0.01610,-0.01610,...,-135.0,4.192,0.137,-0.125,1.499,0.272,-0.245,289.92145,46.744560,12.805
1410,0,0,0,0,7.560522,0.000026,-0.000026,134.47889,0.00270,-0.00270,...,-211.0,4.503,0.052,-0.208,0.940,0.282,-0.094,283.84515,44.609089,15.986
5865,0,0,0,0,4.644901,0.000038,-0.000038,133.67436,0.00786,-0.00786,...,-85.0,4.540,0.052,-0.017,0.770,0.027,-0.046,282.34305,48.340778,14.480
340,0,0,1,1,2.037441,0.000010,-0.000010,133.59962,0.00390,-0.00390,...,-207.0,4.415,0.087,-0.203,1.015,0.312,-0.134,295.79526,47.663960,14.187


# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [7]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [8]:
X_test_scaled

array([[0.        , 0.        , 1.        , ..., 0.36360643, 0.42137677,
        0.68104802],
       [0.        , 0.        , 0.        , ..., 0.5264331 , 0.06595862,
        0.54516902],
       [0.        , 1.        , 0.        , ..., 0.74319829, 0.89120537,
        0.54277213],
       ...,
       [1.        , 0.        , 0.        , ..., 0.71562838, 0.15381737,
        0.5069014 ],
       [0.        , 0.        , 0.        , ..., 0.63000799, 0.5857759 ,
        0.71270353],
       [0.        , 0.        , 0.        , ..., 0.25054006, 0.28680825,
        0.66732788]])

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [10]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [11]:
encoded_y_train

array([2, 0, 1, ..., 0, 0, 0])

In [12]:
y_train_categorical.shape

(6558, 3)

In [13]:
y_train_categorical

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [14]:
y_test_categorical

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

# Create and train the Deep Learning Model using Keras

In [15]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu',))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
# compile and fit the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
6558/6558 - 1s - loss: 0.5279 - acc: 0.7414
Epoch 2/60
6558/6558 - 1s - loss: 0.3752 - acc: 0.8076
Epoch 3/60
6558/6558 - 1s - loss: 0.3555 - acc: 0.8222
Epoch 4/60
6558/6558 - 1s - loss: 0.3458 - acc: 0.8297
Epoch 5/60
6558/6558 - 1s - loss: 0.3395 - acc: 0.8370
Epoch 6/60
6558/6558 - 1s - loss: 0.3326 - acc: 0.8425
Epoch 7/60
6558/6558 - 1s - loss: 0.3213 - acc: 0.8507
Epoch 8/60
6558/6558 - 0s - loss: 0.3188 - acc: 0.8545
Epoch 9/60
6558/6558 - 1s - loss: 0.3097 - acc: 0.8580
Epoch 10/60
6558/6558 - 0s - loss: 0.3036 - acc: 0.8637
Epoch 11/60
6558/6558 - 0s - loss: 0.3036 - acc: 0.8611
Epoch 12/60
6558/6558 - 1s - loss: 0.3026 - acc: 0.8638
Epoch 13/60
6558/6558 - 1s - loss: 0.2996 - acc: 0.8683
Epoch 14/60
6558/6558 - 1s - loss: 0.2929 - acc: 0.8707
Epoch 15/60
6558/6558 - 1s - loss: 0.2953 - acc: 0.8673
Epoch 16/60
6558/6558 - 1s - loss: 0.2873 - acc: 0.8719
Epoch 17/60
6558/6558 - 0s - loss: 0.2847 - acc: 0.8744
Epoch 18/60
6558/6558 - 1s - loss: 0.2851 - acc: 0.8760
E

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2
)
print(f"Normal Neural Network - Loss: {model_loss}\nAccuracy: {model_accuracy}")

2186/2186 - 0s - loss: 0.2661 - acc: 0.8838
Normal Neural Network - Loss: 0.26611795332981486
Accuracy: 0.883806049823761


In [20]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [21]:
print(f"Predicted classes: {prediction_labels[:15]}")
print(f"Actual Labels: {list(y_test[:15])}")

Predicted classes: ['False_Positive' 'CONFIRMED' 'False_Positive' 'False_Positive'
 'CONFIRMED' 'False_Positive' 'False_Positive' 'CONFIRMED' 'CONFIRMED'
 'CANDIDATE' 'False_Positive' 'CONFIRMED' 'CONFIRMED' 'CONFIRMED'
 'CONFIRMED']
Actual Labels: ['False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE', 'False_Positive', 'CONFIRMED', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE']


In [22]:
! pip install joblib


In [23]:
import joblib
filename = 'deep_learning.sav'
joblib.dump(prediction_labels, filename)

['deep_learning.sav']